In [12]:
import cadquery as cq
from cadquery import exporters
from IPython.display import SVG, DisplayObject

# pts = [
#     (-20, -20),
#     (20, 0),
#     (20, 20),
#     (-20, -20),
# ]
# result = cq.Workplane("front").polyline(pts).extrude(20)

(L, H, W, t) = (100.0, 20.0, 20.0, 1.0)
pts = [
    (0, 10),
    (10, -10),
    (0, -10),
]
result = cq.Workplane("front").polyline(pts).mirrorY()

# print each edge
for edge in result.edges().vals():
    for vert in edge.vertices():
        print(vert.center())


# exporters.export(result, 'tmp.stl')

# display(SVG(result.toSvg()))


((0.0, 0.0, 0.0), (3.141592653589793, 0.0, 3.141592653589793))
((0.0, 0.0, 0.0), (0.0, -0.0, 0.0))
((0.0, 0.0, 0.0), (0.0, -0.0, 0.0))
((0.0, 0.0, 0.0), (0.0, -0.0, 0.0))
((0.0, 0.0, 0.0), (0.0, -0.0, 0.0))
((0.0, 0.0, 0.0), (3.141592653589793, 0.0, 3.141592653589793))
((0.0, 0.0, 0.0), (3.141592653589793, 0.0, 3.141592653589793))
((0.0, 0.0, 0.0), (3.141592653589793, 0.0, 3.141592653589793))
